In [1]:
import pandas as pd
import time
from typing import Literal
from functools import partial
from typing import Optional
import os
from tqdm import tqdm

In [2]:
df = pd.read_pickle("./data/data.pkl.gz")
df

,text,uuid,int_0,int_1,int_2,int_3,int_4,int_5,int_6,int_7,...,float_10,float_11,float_12,float_13,float_14,float_15,float_16,float_17,float_18,float_19
0,"Write an article based on this ""A man has been...",9b193ab1-ab92-4d74-9aac-876f21c302a0,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
1,Answer the following question: - number is 54 ...,dc0a6127-3977-4ec2-8ecc-60de9df14571,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
2,Produce a long descriptive sentence that uses ...,b6189a7f-9bd2-4360-b381-6e8bd1056109,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
3,Write a title for this article:\n\nArbitration...,c64c0f95-661e-4ad5-ad61-ab99ecf30f42,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
4,Read the following paragraph and determine if ...,f516250e-57b9-46a4-9397-adbf0c1f5326,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363486,"Q: Context: Dragon Ball Z (ドラゴンボール ゼット, Dorago...",8ba7f794-07f2-451c-90c5-24f7c4e7734b,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
363487,My question is: At the beginning of an academi...,3c338e45-3525-4dd4-88fe-1cc31ccc59ef,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
363488,"Leo: Given the sentence ""A small child in wate...",28a3dbab-0e47-4a40-acc2-05a92f02abb5,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414
363489,Explain simply why yes is the correct answer t...,314b76cb-9390-4e76-8a6a-6d5034765345,3474,-8309,-1183,-6862,-2121,-6685,-6140,-5582,...,-1704.235408,8690.18989,7636.103984,-5305.07615,-4549.317322,-3380.911654,-4222.91801,1709.491323,1727.529697,-6372.93414


In [3]:
# total memory usage in bytes without index
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html
mem_usage = df.memory_usage(index=False, deep=True).sum()
mem_usage

np.int64(823608914)

In [4]:
# total memory usage in m_bytes without index
mem_size_mb = mem_usage / 1024 / 1024
mem_size_mb

np.float64(785.454668045044)

In [5]:
def evaluate_performance(
        df,
        method: Literal["csv", "parquet"],
        compression: Optional[str] = None,
        compression_level: Optional[int] = None,
    ) -> dict:
    filename = f"./test_data/data.{method}"

    # set save_fn and read_fn
    save_fn = None
    read_fn = None
    additional_save_params = {}
    if method == "csv":
        if compression is not None:
            additional_save_params["compression"] = compression
        assert compression_level is None
        save_fn = partial(df.to_csv, index=False, path_or_buf=filename, **additional_save_params)
        read_fn = partial(pd.read_csv, filepath_or_buffer=filename, **additional_save_params)
    elif method == "parquet":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_parquet, index=False, engine="pyarrow", path=filename, **additional_save_params)
        read_fn = partial(pd.read_parquet, path=filename, engine="pyarrow")
    elif method == "feather":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_feather, path=filename, **additional_save_params)
        read_fn = partial(pd.read_feather, path=filename)
    assert save_fn is not None
    assert read_fn is not None


    # write
    start_time = time.time()
    save_fn()
    save_time = time.time() - start_time
    # print("save_time:", save_time)

    # read
    start_time = time.time()
    for _ in range(10):
        read_fn()
    read_time = (time.time() - start_time) / 10
    # print("read_time:", read_time)

    # get size
    file_size = os.path.getsize(filename) / 1024 / 1024
    # print("file_size:", file_size)

    # delete file
    os.remove(filename)

    result = {
        "method": method,
        "save_time": save_time,
        "read_time": read_time,
        "file_size": file_size,
    }

    for k, v in additional_save_params.items():
        result[k] = v

    return result


In [6]:
eval_results = []

In [7]:
# parquet, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:11<03:34, 11.31s/it]

{'method': 'parquet', 'save_time': 1.7168431282043457, 'read_time': 0.9589344024658203, 'file_size': 205.0951271057129, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:23<03:32, 11.80s/it]

{'method': 'parquet', 'save_time': 2.1638808250427246, 'read_time': 0.9968081951141358, 'file_size': 190.3500156402588, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:36<03:26, 12.15s/it]

...

{'method': 'parquet', 'save_time': 77.91794109344482, 'read_time': 0.9199609756469727, 'file_size': 151.67846584320068, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [10:36<01:12, 72.46s/it]

{'method': 'parquet', 'save_time': 86.38836717605591, 'read_time': 0.9317858934402465, 'file_size': 151.54985809326172, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [12:12<00:00, 36.64s/it]

{'method': 'parquet', 'save_time': 86.35913395881653, 'read_time': 0.9622390031814575, 'file_size': 151.54985809326172, 'compression': 'zstd', 'compression_level': 20}


In [8]:
# parquet, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:08<02:32,  8.02s/it]

{'method': 'parquet', 'save_time': 1.338258981704712, 'read_time': 0.6679298162460328, 'file_size': 299.8812093734741, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:15<02:21,  7.84s/it]

{'method': 'parquet', 'save_time': 1.3117198944091797, 'read_time': 0.6393234252929687, 'file_size': 299.8812093734741, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:27<02:43,  9.64s/it]

...

{'method': 'parquet', 'save_time': 28.956976175308228, 'read_time': 0.6442868947982788, 'file_size': 217.89891529083252, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [07:55<00:34, 34.74s/it]

{'method': 'parquet', 'save_time': 28.874755859375, 'read_time': 0.6422530174255371, 'file_size': 217.89891529083252, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [08:30<00:00, 25.54s/it]

{'method': 'parquet', 'save_time': 28.945737838745117, 'read_time': 0.6439600944519043, 'file_size': 217.89891529083252, 'compression': 'lz4', 'compression_level': 20}


In [9]:
# feather, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:08<02:50,  8.95s/it]

{'method': 'feather', 'save_time': 1.3659017086029053, 'read_time': 0.758134388923645, 'file_size': 203.28154182434082, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:19<02:54,  9.71s/it]

{'method': 'feather', 'save_time': 1.9289109706878662, 'read_time': 0.8319602966308594, 'file_size': 188.03924751281738, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:29<02:51, 10.08s/it]

...

{'method': 'feather', 'save_time': 146.68037581443787, 'read_time': 0.74114511013031, 'file_size': 129.81019020080566, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [15:53<02:11, 131.66s/it]

{'method': 'feather', 'save_time': 182.27556467056274, 'read_time': 0.750898814201355, 'file_size': 128.0803165435791, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [19:48<00:00, 59.42s/it] 

{'method': 'feather', 'save_time': 227.16688799858093, 'read_time': 0.7431152105331421, 'file_size': 119.38253974914551, 'compression': 'zstd', 'compression_level': 20}


In [10]:
# feather, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:06<02:02,  6.42s/it]

{'method': 'feather', 'save_time': 1.2404112815856934, 'read_time': 0.5180318117141723, 'file_size': 299.81385231018066, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:12<01:54,  6.38s/it]

{'method': 'feather', 'save_time': 1.2343418598175049, 'read_time': 0.5106603860855102, 'file_size': 299.81385231018066, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:23<02:19,  8.19s/it]

...

{'method': 'feather', 'save_time': 30.854987144470215, 'read_time': 0.5146545886993408, 'file_size': 217.457124710083, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [07:47<00:35, 35.28s/it]

{'method': 'feather', 'save_time': 30.856679916381836, 'read_time': 0.5138850927352905, 'file_size': 217.457124710083, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [08:23<00:00, 25.16s/it]

{'method': 'feather', 'save_time': 30.903788089752197, 'read_time': 0.5150339126586914, 'file_size': 217.457124710083, 'compression': 'lz4', 'compression_level': 20}


In [11]:
# parquet
for c in ["gzip", "snappy", "lz4", "zstd", "brotli", None]:
    r = evaluate_performance(df, "parquet", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'parquet', 'save_time': 19.627086877822876, 'read_time': 1.7965444087982179, 'file_size': 189.7540044784546, 'compression': 'gzip'}
{'method': 'parquet', 'save_time': 1.3376078605651855, 'read_time': 0.7331516027450562, 'file_size': 296.9213275909424, 'compression': 'snappy'}
{'method': 'parquet', 'save_time': 1.3070762157440186, 'read_time': 0.6384718894958497, 'file_size': 299.8812093734741, 'compression': 'lz4'}
{'method': 'parquet', 'save_time': 1.483922004699707, 'read_time': 0.8763885021209716, 'file_size': 205.0951271057129, 'compression': 'zstd'}
{'method': 'parquet', 'save_time': 18.214473962783813, 'read_time': 1.545258593559265, 'file_size': 159.6326036453247, 'compression': 'brotli'}
{'method': 'parquet', 'save_time': 1.3652260303497314, 'read_time': 0.7113039016723632, 'file_size': 296.9213275909424}


In [12]:
# feather
for c in ["lz4", "zstd", "uncompressed", None]:
    r = evaluate_performance(df, "feather", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'feather', 'save_time': 1.2270021438598633, 'read_time': 0.5096915006637573, 'file_size': 299.81385231018066, 'compression': 'lz4'}
{'method': 'feather', 'save_time': 1.355456829071045, 'read_time': 0.7567508935928344, 'file_size': 203.28154182434082, 'compression': 'zstd'}
{'method': 'feather', 'save_time': 0.33922886848449707, 'read_time': 0.38898358345031736, 'file_size': 634.9402027130127, 'compression': 'uncompressed'}
{'method': 'feather', 'save_time': 1.2628107070922852, 'read_time': 0.5181000232696533, 'file_size': 299.81385231018066}


In [13]:
# csv
for c in ['zip', 'gzip', 'bz2', 'zstd', 'xz', 'tar', None]:
    r = evaluate_performance(df, "csv", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'csv', 'save_time': 27.699738264083862, 'read_time': 5.631162214279175, 'file_size': 199.11504650115967, 'compression': 'zip'}
{'method': 'csv', 'save_time': 31.161946773529053, 'read_time': 7.01388680934906, 'file_size': 198.55400371551514, 'compression': 'gzip'}
{'method': 'csv', 'save_time': 62.240323066711426, 'read_time': 16.672200083732605, 'file_size': 153.71932125091553, 'compression': 'bz2'}
{'method': 'csv', 'save_time': 12.84618878364563, 'read_time': 5.495090508460999, 'file_size': 184.9435920715332, 'compression': 'zstd'}
{'method': 'csv', 'save_time': 211.0541648864746, 'read_time': 11.23839750289917, 'file_size': 131.52967834472656, 'compression': 'xz'}
{'method': 'csv', 'save_time': 10.89088487625122, 'read_time': 4.963031196594239, 'file_size': 691.806640625, 'compression': 'tar'}
{'method': 'csv', 'save_time': 10.828938961029053, 'read_time': 4.969729208946228, 'file_size': 691.7978973388672}


In [14]:
df = pd.DataFrame(eval_results)
df

,method,save_time,read_time,file_size,compression,compression_level
0,parquet,1.716843,0.958934,205.095127,zstd,1.0
1,parquet,2.163881,0.996808,190.350016,zstd,2.0
2,parquet,2.461648,1.010983,181.888956,zstd,3.0
3,parquet,2.551957,1.007971,179.640458,zstd,4.0
4,parquet,4.067563,1.028516,174.038219,zstd,5.0
...,...,...,...,...,...,...
92,csv,62.240323,16.672200,153.719321,bz2,NaN
93,csv,12.846189,5.495091,184.943592,zstd,NaN
94,csv,211.054165,11.238398,131.529678,xz,NaN
95,csv,10.890885,4.963031,691.806641,tar,NaN


In [15]:
df["comp_ratio"] = mem_size_mb / df["file_size"]
df

,method,save_time,read_time,file_size,compression,compression_level,comp_ratio
0,parquet,1.716843,0.958934,205.095127,zstd,1.0,3.829709
1,parquet,2.163881,0.996808,190.350016,zstd,2.0,4.126370
2,parquet,2.461648,1.010983,181.888956,zstd,3.0,4.318320
3,parquet,2.551957,1.007971,179.640458,zstd,4.0,4.372371
4,parquet,4.067563,1.028516,174.038219,zstd,5.0,4.513116
...,...,...,...,...,...,...,...
92,csv,62.240323,16.672200,153.719321,bz2,NaN,5.109668
93,csv,12.846189,5.495091,184.943592,zstd,NaN,4.246996
94,csv,211.054165,11.238398,131.529678,xz,NaN,5.971692
95,csv,10.890885,4.963031,691.806641,tar,NaN,1.135367


In [16]:
df.to_pickle("./data/test_results_02.pkl.gz", compression="gzip")